In [1]:
import numpy as np
import pandas as pd
import matplotlib         
import matplotlib.pyplot as plt       
import pykat.optics.maps as pkm
from pykat import finesse                 
from pykat.commands import *               
from pykat.optics.maps import *            
from IPython.display import display, HTML
from scipy.special import factorial
import fileinput

%matplotlib inline
pykat.init_pykat_plotting(dpi=90)

                                              ..-
    PyKat 1.2.1           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



In [2]:
# return a list of zernike maps(basis) for aLIGO maps to project to
def Zernikes(shape, radius, step_size, max_zern):# size leads to the cropping radius 
    center = round(shape-1)/2
    rrange = round(radius/step_size)
    zernikes=[]
    def theta(x,y):
        return np.where(x == 0, np.where(y>=0,np.pi/2,-np.pi/2), np.where(x>0,np.arctan(y/x),np.arctan(y/x)+np.pi))

    def radial(x,y,n,m):
        if m<0:
            m=-m
        sum=0
        for k in range(int((n-m)/2)+1):   
            r=(-1)**k*factorial(n-k)/factorial(k)/factorial((n+m)/2-k)/factorial((n-m)/2-k)*((x**2+y**2)/(385**2))**(n/2-k)
            sum+=r
        return sum

    def angular(x,y,n,m): 
        a=theta(x,y)
        if m>=0:
            angular=np.cos(m*a)
        else:
            angular=-np.sin(m*a)
        return angular
    
    for n in range(max_zern):
        for m in range(-n,n+1,2):
            stepRange = np.arange(shape)-center
            x,y=np.meshgrid(stepRange,stepRange,sparse=True)
            zfunc=radial(x,y,n,m)*angular(x,y,n,m)
            for i in range(shape):
                for j in range(shape): 
                    if (i-center)**2+(j-center)**2>= rrange**2:
                        zfunc[i][j]=0
            zmap=zfunc/zfunc.max()
            zernikes.append(zmap)
    return zernikes

In [4]:
def layerCoeffs(filename, zernikebasis, order=10):
    i=0# create a conversion between {n}{m} to {i}
    dic={}
    for n in range(order):
        for m in range(-n,n+1,2):
            dic[f'{n}{m}']=i
            i=i+1

    aLIGO=pd.read_csv(filename, header=None, sep=" ", skiprows=9).dropna(axis=1).values
    
    overlap=[]
    zernikes=[]
    layerCoeff=[]
    amap=0
    for n in range(order):
        layer=0
        for m in range(-n,n+1,2):
            index=dic[f'{n}{m}']
            coeff=(aLIGO*zernikebasis[index]).sum()/((zernikebasis[index]**2).sum())
 #           print(f"A{n}{m} is {coeff}")
            amap+=zernikebasis[index]*coeff
            overlap.append(coeff)
            if n>=3:
                layer+=coeff**2
            elif n==2:
                if m==-2 or m==2:
                    layer+=coeff**2
            else:
                layer=0         
        layerCoeff.append(np.sqrt(layer))
    return np.array(layerCoeff)

In [6]:
layerCoeffs('aLIGOmeasuredmaps/ETM08_S1_-power160_finesse.txt',Zernikes(777,0.154,0.0004000000189989805,10))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


A00 is -11.888554902659674
A1-1 is 0.7602347312977585
A11 is 0.19086576746444306
A2-2 is 0.7000131634612041
A20 is -22.89882222802464
A22 is 1.0727560429532612
A3-3 is 0.7755178477453356
A3-1 is 0.32748919121671605
A31 is -0.08446471321633488
A33 is -0.7501310678036963
A4-4 is 0.03147061522247439
A4-2 is 0.40881240825423343
A40 is -13.314920336760043
A42 is 0.06749896450608868
A44 is -0.6136663598018668
A5-5 is 0.3778127240485924
A5-3 is 0.09514262946627805
A5-1 is -0.3341574428439815
A51 is -0.5765120599411635
A53 is 0.14292781718358818
A55 is 0.017134698566240805
A6-6 is -0.03710079586443687
A6-4 is -0.03559632969369133
A6-2 is 0.4790542339074026
A60 is -2.2594791368547376
A62 is -0.26260613360113194
A64 is 0.013238824634700863
A66 is -0.1831008765570378
A7-7 is -0.8251240710586916
A7-5 is 0.1761008895467249
A7-3 is -0.09856739311575169
A7-1 is -0.49279507282018137
A71 is -0.8159788167173727
A73 is 0.09092744631049146
A75 is -0.015778360156376056
A77 is -0.34172698001509144
A8-8 is 0

In [6]:
coeff01=layerCoeffs('aLIGO measured maps/xITM_09_R2_Figure_finesse.txt',Zernikes(1133,0.15,0.0004000000189989805,10))
coeff01

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.44287268, 0.29085334, 0.27624584,
       0.14614646, 0.09594378, 0.20533683, 0.26391068, 0.08582543])

In [12]:
coeff02=layerCoeffs('aLIGO measured maps/ETM05_S1_finesse.txt',Zernikes(1131,0.15,0.00026699510635808, 10))
coeff02

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.20186108, 0.37465685, 0.6104358 ,
       0.44928513, 0.69326921, 0.49323309, 0.7545034 , 0.52067729])

In [13]:
coeff03=layerCoeffs('aLIGO measured maps/ETM08_S1_-power160_finesse.txt',Zernikes(777,0.154,0.0004000000189989805,10))
coeff03

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([ 0.        ,  0.        ,  1.28094651,  1.13071128, 13.33553014,
        0.78520309,  2.33239055,  1.32496011,  1.0215447 ,  1.16137429])

In [17]:
coeff04=layerCoeffs('aLIGO measured maps/itm08_s2_finesse.txt',Zernikes(1133,0.15,0.0002668091910891235,10))
coeff04

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.71844138, 1.29780436, 2.210698  ,
       1.46256743, 2.49307016, 1.56083758, 2.67519769, 1.62018873])

In [18]:
coeff05=layerCoeffs('aLIGO measured maps/xETM_07_R1_Figure_finesse.txt',Zernikes(1131,0.15,0.0002669949899427593,10))
coeff05

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.15035322, 1.24354955, 0.3672203 ,
       1.46105566, 0.45210691, 1.6028785 , 0.52205935, 1.70091066])

In [19]:
coeff06=layerCoeffs('aLIGO measured maps/itm04_s2_finesse.txt',Zernikes(1133,0.15,0.00026675150729715824,10))
coeff06

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 4.8451721 , 2.18825039, 5.65517414,
       2.65662153, 5.94070816, 2.94367889, 6.1293699 , 3.14660087])

In [20]:
coeff07=layerCoeffs('aLIGO measured maps/xITM_07_R1_Figure_finesse.txt',Zernikes(1129,0.15,0.00026716801221482456,10))
coeff07

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.41626056, 1.19931404, 0.45847125,
       1.45809959, 0.49255718, 1.61986629, 0.51873962, 1.73074644])

In [21]:
coeff08=layerCoeffs('aLIGO measured maps/xITM_07_R2_Figure_finesse.txt',Zernikes(1133,0.15,0.0002668089873623103,10))
coeff08

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 5.45324274, 6.59631406, 7.09182587,
       7.91408725, 7.84028778, 8.67276644, 8.2099705 , 9.09805661])

In [22]:
coeff09=layerCoeffs('aLIGO measured maps/xITM_07_SPTWE_Figure_finesse.txt',Zernikes(1133,0.15,0.0002668089873623103,10))
coeff09

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([ 0.        ,  0.        , 12.77427359, 28.51925341, 41.85681877,
       36.87917958, 51.25676881, 41.14469415, 58.11421187, 43.8935028 ])

In [23]:
coeff10=layerCoeffs('aLIGO measured maps/xITM_11_R1_Figure_finesse.txt',Zernikes(1133,0.15,0.0002667510125320405,10))
coeff10

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.39514777, 1.15570595, 0.4940704 ,
       1.31703502, 0.54923939, 1.4199425 , 0.60010472, 1.49449418])

In [24]:
coeff11=layerCoeffs('aLIGO measured maps/xITM_11_SPTWE_Figure_finesse.txt',Zernikes(1133,0.15,0.0002668089873623103,10))
coeff11

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([  0.        ,   0.        ,  30.4624681 ,  36.69004012,
       398.86272698,  45.9948354 , 480.96218103,  50.95691893,
       542.1680594 ,  54.11547618])

In [25]:
coeff12=layerCoeffs('aLIGO measured maps/xITM_03_R1_Figure_finesse.txt',Zernikes(1133,0.15,0.0002667510125320405,10))
coeff12

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.05451099, 1.34037573, 0.20983636,
       1.61458082, 0.33692235, 1.78815292, 0.43619608, 1.90731806])

In [26]:
coeff13=layerCoeffs('aLIGO measured maps/xITM_03_R2_Figure_finesse.txt',Zernikes(1133,0.15,0.0002668089873623103,10))
coeff13

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 1.32869277, 3.18272895, 5.86101388,
       3.9358462 , 6.71906015, 4.39359299, 7.2844799 , 4.7070334 ])

In [27]:
coeff14=layerCoeffs('aLIGO measured maps/xITM_05_R2_Figure_finesse.txt',Zernikes(1133,0.15,0.0002668089873623103,10))
coeff14

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([ 0.        ,  0.        ,  9.75019755,  7.31657684, 12.64987283,
        8.37711906, 13.88123751,  8.93722022, 14.53867507,  9.23267667])

In [28]:
coeff15=layerCoeffs('aLIGO measured maps/xITM_11_R2_Figure_finesse.txt',Zernikes(1133,0.15,0.0002668089873623103,10))
coeff15

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 4.72189406, 2.64984872, 6.88477325,
       3.41606407, 7.41277184, 3.77731604, 7.67932514, 3.99741562])

In [29]:
coeff16=layerCoeffs('aLIGO measured maps/itm04_s1_finesse.txt',Zernikes(1133,0.15,0.00026675150729715824,10))
coeff16

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.57241227, 1.13432651, 0.98601743,
       1.43530455, 1.16614131, 1.62416901, 1.27820446, 1.75060576])

In [31]:
coeff17=layerCoeffs('aLIGO measured maps/xITM_06_R2_Figure_finesse.txt',Zernikes(1133,0.15,0.0002668089873623103,10))
coeff17

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.25181065, 0.92148281, 0.95587634,
       1.01575246, 0.98557096, 1.0619721 , 0.99351183, 1.08864545])

In [32]:
coeff18=layerCoeffs('aLIGO measured maps/xITM_09_R1_Figure_finesse.txt',Zernikes(1133,0.15,0.0002667510125320405,10))
coeff18

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.39528349, 0.7269017 , 0.86217052,
       0.85022691, 0.98472151, 0.92681821, 1.05754283, 0.98154609])

In [33]:
coeff19=layerCoeffs('aLIGO measured maps/xITM_10_R1_Figure_finesse.txt',Zernikes(1133,0.15,0.0002667510125320405,10))
coeff19

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.24387631, 2.67198784, 0.53458099,
       3.20934947, 0.53416862, 3.54031538, 0.53560118, 3.77192861])

In [34]:
coeff20=layerCoeffs('aLIGO measured maps/xITM_10_R2_Figure_finesse.txt',Zernikes(1133,0.15,0.0002668089873623103,10))
coeff20

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([ 0.        ,  0.        ,  6.00232723,  3.24153954, 11.35248949,
        3.6398263 , 13.38371346,  3.82966677, 14.77982948,  3.9318195 ])

In [4]:
coeff21=layerCoeffs('aLIGO measured maps/itm08_s1_finesse.txt',Zernikes(1133,0.15,0.00026675150729715824,10))
coeff21

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.24134709, 0.56735222, 0.56657343,
       0.73135515, 0.63527991, 0.85265032, 0.67373944, 0.93413568])

In [5]:
coeff22=layerCoeffs('aLIGO measured maps/ETM02-S1_finesse.txt',Zernikes(1131,0.15,0.0002669951063580811,10))
coeff22

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


array([0.        , 0.        , 0.76808349, 1.18982281, 0.97097653,
       1.40479173, 1.08079276, 1.51390917, 1.12898421, 1.57590949])

In [ ]:
layerarray = np.zeros((20,10))
for i in range(1,21):
    if i<10:
        layerarray[i-1]=eval('coeff0' + str(i))
    else:
        layerarray[i-1]=eval('coeff' + str(i))

pd.DataFrame(layerarray).to_pickle('pkl/layerArray_20maps.pkl')
layerarray=pd.read_pickle('pkl/layerArray_20maps.pkl').values

layerarray22 = np.concatenate((layerarray, coeff21.reshape(1,10), coeff22.reshape(1,10)), axis=0)
pd.DataFrame(layerarray22).to_pickle('pkl/layerArray_22maps.pkl')

In [3]:
layerDF

,Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9
0,0.0,0.0,0.442873,0.290853,0.276246,0.146146,0.095944,0.205337,0.263911,0.085825
1,0.0,0.0,0.201861,0.374657,0.610436,0.449285,0.693269,0.493233,0.754503,0.520677
2,0.0,0.0,1.280947,1.130711,13.335530,0.785203,2.332391,1.324960,1.021545,1.161374
3,0.0,0.0,0.718441,1.297804,2.210698,1.462567,2.493070,1.560838,2.675198,1.620189
4,0.0,0.0,0.150353,1.243550,0.367220,1.461056,0.452107,1.602879,0.522059,1.700911
5,0.0,0.0,4.845172,2.188250,5.655174,2.656622,5.940708,2.943679,6.129370,3.146601
6,0.0,0.0,0.416261,1.199314,0.458471,1.458100,0.492557,1.619866,0.518740,1.730746
7,0.0,0.0,0.395148,1.155706,0.494070,1.317035,0.549239,1.419942,0.600105,1.494494
8,0.0,0.0,0.054511,1.340376,0.209836,1.614581,0.336922,1.788153,0.436196,1.907318
9,0.0,0.0,1.328693,3.182729,5.861014,3.935846,6.719060,4.393593,7.284480,4.707033


In [2]:
# find the statistical distributions of each layers
# indexes=list(range(20))
# indexes.remove(8)
# indexes.remove(10)
layerarray22=pd.read_pickle('pkl/layerArray_22maps.pkl').values
layerDF=pd.DataFrame(np.delete(layerarray22, [7,8,10,13,19], 0), columns=[f'Z{i}' for i in range(10)])
stats=layerDF.describe()
stats

,Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9
count,17.0,17.0,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000
mean,0.0,0.0,1.001704,1.368569,2.425863,1.608781,1.935571,1.802919,1.973707,1.892668
std,0.0,0.0,1.468061,0.827093,3.550311,1.073051,2.369902,1.173663,2.488309,1.273487
min,0.0,0.0,0.054511,0.290853,0.209836,0.146146,0.095944,0.205337,0.263911,0.085825
25%,0.0,0.0,0.243876,0.921483,0.494070,0.850227,0.534169,1.061972,0.535601,1.088645
50%,0.0,0.0,0.416261,1.189823,0.862171,1.435305,0.984722,1.560838,0.993512,1.620189
75%,0.0,0.0,0.768083,1.340376,2.210698,1.614581,2.332391,1.788153,1.278204,1.907318
max,0.0,0.0,4.845172,3.182729,13.335530,3.935846,7.412772,4.393593,7.679325,4.707033


In [4]:
# use the above statistics to generate the corresponding gaussian distribution
mu, sigma = stats.loc['mean'], stats.loc['std']
coeff_random=np.zeros((50,10))
for i in range(50):
    for j in range(10):
        coeff_random[i][j] = np.random.normal(mu[j], sigma[j], 1)

In [5]:
coeff_random[coeff_random<0] = 0
coeff_random

array([[0.00000000e+00, 0.00000000e+00, 2.14355525e+00, 1.25403439e+00,
        6.51080701e+00, 9.38473337e-01, 0.00000000e+00, 1.36933666e+00,
        1.61377746e+00, 3.78581112e+00],
       [0.00000000e+00, 0.00000000e+00, 1.27363035e+00, 6.71970278e-01,
        3.23789214e+00, 2.51515299e-01, 0.00000000e+00, 1.58409656e+00,
        4.18881397e+00, 2.13154997e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.46645751e+00,
        1.82445969e+00, 1.68101794e+00, 3.37273932e-01, 0.00000000e+00,
        4.94765949e+00, 9.48285875e-01],
       [0.00000000e+00, 0.00000000e+00, 1.09818052e+00, 2.21523733e+00,
        3.62895119e+00, 1.60775128e+00, 0.00000000e+00, 9.25579105e-01,
        6.48557589e+00, 4.70350127e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.45380470e-01,
        2.35307025e+00, 2.92766164e+00, 8.13848547e+00, 2.02471656e+00,
        4.38580485e+00, 1.63579812e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.89987795e+00,
   

In [6]:
pd.DataFrame(coeff_random).to_pickle('pkl/random_Coeffs.pkl')